# Rossman Pharmaceutical Sales Prediction

## Imports

In [7]:
import numpy as np
import pandas as pd
import sys
import mlflow
import warnings
import pickle
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import ticker
from statsmodels.tsa.stattools import adfuller, acf, pacf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [8]:
# sns.set()
%matplotlib inline
warnings.filterwarnings("ignore")

In [13]:
sys.path.append("/Users/user/Pharmaceutical_Sales_prediction")
#from scripts import file_handler

## Preprocessing

We have alreadyseen which features influence sales in the exploratory section. And we have used many of the columns for the 4 machine learning models used as seen in the scripts. But for LSTM only the date and sales columns will be used.

In [17]:
#file_handler = FileHandler()
#train_sales = file_handler.read_csv("../data/train.csv")
train_file = '../data/clean_train_store.csv'
train_sales = pd.read_csv(train_file, parse_dates = True, low_memory = False)

In [18]:
data = train_sales.groupby("Date").agg({"Sales": "mean"})
data

,Sales
Date,
2013-01-01,18.755556
2013-01-02,5991.964444
2013-01-03,5400.386667
2013-01-04,5716.217778
2013-01-05,5104.604444
...,...
2015-07-27,9330.057522
2015-07-28,7892.123894
2015-07-29,7428.000000


In [19]:
scaler = StandardScaler()
scaled_array = scaler.fit_transform(data)
data['DataScaled'] = scaled_array


In [20]:
SIZE = len(data.DataScaled)
WINDOW_SIZE = 48
BATCH_SIZE = SIZE - WINDOW_SIZE * 2
EPOCHS = 200

In [21]:
DateTrain = data.index.values[0:BATCH_SIZE]
DateValid = data.index.values[BATCH_SIZE:]
XTrain = data.DataScaled.values[0:BATCH_SIZE].astype('float32')
XValid = data.DataScaled.values[BATCH_SIZE:].astype('float32')

# Obtain shapes for vectors of size (,1) for dates series

DateTrain = np.reshape(DateTrain, (-1, 1))
DateValid = np.reshape(DateValid, (-1, 1))

print("Shape of the training set date series: ", DateTrain.shape)
print("Shape of the validation set date series: ", DateValid.shape)
print()
print("Shape of the training set logarithm of sales series: ", XTrain.shape)
print("Shape of the validation set logarithm of sales series in a stateless LSTM: ", XValid.shape)


Shape of the training set date series:  (846, 1)
Shape of the validation set date series:  (96, 1)

Shape of the training set logarithm of sales series:  (846,)
Shape of the validation set logarithm of sales series in a stateless LSTM:  (96,)


In [22]:
tf.random.set_seed(1234)
# add extra dimension
series = tf.expand_dims(XTrain, axis=-1)
series.shape


TensorShape([846, 1])

In [23]:
# create tensor from each individual element
dataset = tf.data.Dataset.from_tensor_slices(series)
dataset


<TensorSliceDataset element_spec=TensorSpec(shape=(1,), dtype=tf.float32, name=None)>

In [24]:
# takes a window_size + 1 chunk from the slices
dataset = dataset.window(WINDOW_SIZE + 1, shift=1, drop_remainder=True)


In [25]:
# Example of Window
datasetEx = tf.data.Dataset.from_tensor_slices(tf.range(10))
datasetEx = datasetEx.window(5, shift=1, drop_remainder=True)
for window in datasetEx:
    print([elem.numpy() for elem in window])


[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


In [26]:
dataset = dataset.flat_map(lambda window: window.batch(WINDOW_SIZE + 1))


In [27]:
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))


In [28]:
dataset = dataset.batch(BATCH_SIZE).prefetch(1)


In [29]:
def windowed_dataset(series, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE):
  series = tf.expand_dims(series, axis=-1)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset


In [30]:
DatasetTrain = windowed_dataset(XTrain)
DatasetVal = windowed_dataset(XValid)


## Model Training

In [31]:
model = Sequential()
model.add(LSTM(8, input_shape=[None, 1], return_sequences=True))
model.add(LSTM(4, input_shape=[None, 1]))
model.add(Dense(1))
model.compile(loss="huber_loss", optimizer='adam')


In [32]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 8)           320       
                                                                 
 lstm_1 (LSTM)               (None, 4)                 208       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 533
Trainable params: 533
Non-trainable params: 0
_________________________________________________________________


In [33]:
mlflow.set_experiment("LSTM")
mlflow.tensorflow.autolog()
history = model.fit(DatasetTrain, epochs=EPOCHS, validation_data=DatasetVal, verbose=1)

Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\mlflow\store\tracking\file_store.py", line 256, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "C:\Users\user\anaconda3\lib\site-packages\mlflow\store\tracking\file_store.py", line 336, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "C:\Users\user\anaconda3\lib\site-packages\mlflow\utils\file_utils.py", line 175, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file 'C:\Users\user\Pharmaceutical_Sales_prediction\notebooks\mlruns\mlruns\meta.yaml' does not exist.


INFO: 'LSTM' does not exist. Creating a new experiment


Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\mlflow\store\tracking\file_store.py", line 256, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "C:\Users\user\anaconda3\lib\site-packages\mlflow\store\tracking\file_store.py", line 336, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "C:\Users\user\anaconda3\lib\site-packages\mlflow\utils\file_utils.py", line 175, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file 'C:\Users\user\Pharmaceutical_Sales_prediction\notebooks\mlruns\mlruns\meta.yaml' does not exist.
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\mlflow\store\tracking\file_store.py", line 256, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "C:\Users\user\anaconda3\lib\site-packages\mlflow\store\t

Epoch 1/200
1/1 [==============================] - 20s 20s/step - loss: 0.4074 - val_loss: 0.4022
Epoch 2/200
1/1 [==============================] - 0s 283ms/step - loss: 0.4066 - val_loss: 0.4008
Epoch 3/200
1/1 [==============================] - 0s 318ms/step - loss: 0.4059 - val_loss: 0.3994
Epoch 4/200
1/1 [==============================] - 0s 321ms/step - loss: 0.4052 - val_loss: 0.3980
Epoch 5/200
1/1 [==============================] - 1s 1s/step - loss: 0.4045 - val_loss: 0.3967
Epoch 6/200
1/1 [==============================] - 1s 1s/step - loss: 0.4038 - val_loss: 0.3953
Epoch 7/200
1/1 [==============================] - 1s 579ms/step - loss: 0.4032 - val_loss: 0.3940
Epoch 8/200
1/1 [==============================] - 1s 1s/step - loss: 0.4025 - val_loss: 0.3927
Epoch 9/200
1/1 [==============================] - 3s 3s/step - loss: 0.4019 - val_loss: 0.3914
Epoch 10/200
1/1 [==============================] - 2s 2s/step - loss: 0.4013 - val_loss: 0.3900
Epoch 11/200
1/1 [=======

Epoch 84/200
1/1 [==============================] - 0s 303ms/step - loss: 0.3828 - val_loss: 0.3508
Epoch 85/200
1/1 [==============================] - 0s 322ms/step - loss: 0.3825 - val_loss: 0.3504
Epoch 86/200
1/1 [==============================] - 0s 288ms/step - loss: 0.3823 - val_loss: 0.3499
Epoch 87/200
1/1 [==============================] - 0s 483ms/step - loss: 0.3820 - val_loss: 0.3495
Epoch 88/200
1/1 [==============================] - 2s 2s/step - loss: 0.3817 - val_loss: 0.3490
Epoch 89/200
1/1 [==============================] - 1s 821ms/step - loss: 0.3814 - val_loss: 0.3484
Epoch 90/200
1/1 [==============================] - 1s 1s/step - loss: 0.3811 - val_loss: 0.3479
Epoch 91/200
1/1 [==============================] - 1s 757ms/step - loss: 0.3808 - val_loss: 0.3473
Epoch 92/200
1/1 [==============================] - 1s 1s/step - loss: 0.3805 - val_loss: 0.3467
Epoch 93/200
1/1 [==============================] - 2s 2s/step - loss: 0.3802 - val_loss: 0.3461
Epoch 94/200

1/1 [==============================] - 0s 305ms/step - loss: 0.3519 - val_loss: 0.3016
Epoch 167/200
1/1 [==============================] - 0s 283ms/step - loss: 0.3512 - val_loss: 0.3007
Epoch 168/200
1/1 [==============================] - 0s 280ms/step - loss: 0.3506 - val_loss: 0.2998
Epoch 169/200
1/1 [==============================] - 0s 306ms/step - loss: 0.3499 - val_loss: 0.2989
Epoch 170/200
1/1 [==============================] - 1s 897ms/step - loss: 0.3492 - val_loss: 0.2980
Epoch 171/200
1/1 [==============================] - 2s 2s/step - loss: 0.3485 - val_loss: 0.2970
Epoch 172/200
1/1 [==============================] - 1s 846ms/step - loss: 0.3478 - val_loss: 0.2961
Epoch 173/200
1/1 [==============================] - 1s 578ms/step - loss: 0.3471 - val_loss: 0.2951
Epoch 174/200
1/1 [==============================] - 1s 1s/step - loss: 0.3464 - val_loss: 0.2941
Epoch 175/200
1/1 [==============================] - 1s 591ms/step - loss: 0.3457 - val_loss: 0.2932
Epoch 176/

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmppqhgya_v\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmppqhgya_v\model\data\model\assets


In [34]:
from time import gmtime, strftime

In [37]:
time = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model.save(f'../models/LSTM_sales-{time}.pkl')

INFO:tensorflow:Assets written to: ../models/LSTM_sales-2022-05-28-13-30-53.pkl\assets


INFO:tensorflow:Assets written to: ../models/LSTM_sales-2022-05-28-13-30-53.pkl\assets
